Create master key
create certificate
backup master key and certificate

In [10]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'P@ssword01'
CREATE CERTIFICATE sql_cert WITH SUBJECT ='sql1'
BACKUP CERTIFICATE sql_cert TO FILE = '/var/opt/mssql/data/sql_cert.cer' 
WITH PRIVATE KEY (FILE = '/var/opt/mssql/data/privatekey.pvk', 
ENCRYPTION BY PASSWORD = 'P@ssword01')

: Msg 15578, Level 16, State 1, Line 1
There is already a master key in the database. Please drop it before performing this statement.

: Msg 15232, Level 16, State 1, Line 2
A certificate with name 'sql_cert' already exists or this certificate already has been added to the database.

: Msg 15240, Level 16, State 1, Line 3
Cannot write into file '/var/opt/mssql/data/privatekey.pvk'. Verify that you have write permissions, that the file path is valid, and that the file does not already exist.

Total execution time: 00:00:00.052

**Copy the cert and key to local drive using terminal execute the docker commands***

sudo docker cp sql1:/var/opt/mssql/data/sql_cert.cer Documents

sudo docker cp sql1:/var/opt/mssql/data/privatekey.pvk Documents

In [2]:
SELECT * FROM sys.certificates
WHERE name = 'sql_cert'


(1 row affected)

Total execution time: 00:00:00.022

name,certificate_id,principal_id,pvt_key_encryption_type,pvt_key_encryption_type_desc,is_active_for_begin_dialog,issuer_name,cert_serial_number,sid,string_sid,subject,expiry_date,start_date,thumbprint,attested_by,pvt_key_last_backup_date,key_length
sql_cert,258,1,MK,ENCRYPTED_BY_MASTER_KEY,1,sql1,db d5 19 3e 42 25 91 51,0x01060000000000090100000027C46828C8C0E0D9502E0ECB336BB6B26172FC46,S-1-9-1-677954599-3655385288-3406704208-2998299443-1190949473,sql1,2021-05-26 00:14:28.000,2020-05-26 00:14:28.000,0x27C46828C8C0E0D9502E0ECB336BB6B26172FC46,NULL,2020-05-26 00:14:28.877,2048


**Copy cert to sql2 in terminal ~ run the command from backup

sudo docker cp Documents/sql_cert.cer sql2:/var/opt/mssql/data/sql_cert.cer 

sudo docker cp Documents/privatekey.pvk sql2:/var/opt/mssql/data/privatekey.pvk 

Create a new certificate from backup


In [5]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'P@ssword01'
CREATE CERTIFICATE sql_cert FROM FILE='/var/opt/mssql/data/sql_cert.cer' WITH
PRIVATE KEY (FILE = '/var/opt/mssql/data/privatekey.pvk',
DECRYPTION BY PASSWORD = 'P@ssword01')

Commands completed successfully.

Total execution time: 00:00:00.045

In [6]:
SELECT * FROM sys.certificates
WHERE name = 'sql_cert'

(1 row affected)

Total execution time: 00:00:00.555

name,certificate_id,principal_id,pvt_key_encryption_type,pvt_key_encryption_type_desc,is_active_for_begin_dialog,issuer_name,cert_serial_number,sid,string_sid,subject,expiry_date,start_date,thumbprint,attested_by,pvt_key_last_backup_date,key_length
sql_cert,258,1,MK,ENCRYPTED_BY_MASTER_KEY,1,sql1,db d5 19 3e 42 25 91 51,0x01060000000000090100000027C46828C8C0E0D9502E0ECB336BB6B26172FC46,S-1-9-1-677954599-3655385288-3406704208-2998299443-1190949473,sql1,2021-05-26 00:14:28.000,2020-05-26 00:14:28.000,0x27C46828C8C0E0D9502E0ECB336BB6B26172FC46,NULL,NULL,2048


[]()Create a new Database on SQL Server


In [ ]:
CREATE DATABASE testdb
ALTER DATABASE testdb SET RECOVERY FULL
BACKUP DATABASE testdb TO DISK = N'/var/opt/mssql/data/testdb.bak';
BACKUP LOG testdb TO DISK =  N'/var/opt/mssql/data/testdb_log.trn';

Create endpoint and start endpoint


In [15]:
CREATE ENDPOINT hadr_endpoint
AS TCP (LISTENER_PORT=5022)
FOR DATABASE_MIRRORING (
    ROLE = ALL,
    AUTHENTICATION = CERTIFICATE sql_cert,
    ENCRyPTION = REQUIRED ALGORITHM AES
);
ALTER ENDPOINT hadr_endpoint STATE = STARTED;

Commands completed successfully.

Total execution time: 00:00:04.041

CREATE AG USING CONTAINER ID

In [17]:
CREATE AVAILABILITY GROUP [ag1]
    WITH (CLUSTER_TYPE = NONE)
    FOR REPLICA ON
        N'ENTER Container ID' WITH (
            ENDPOINT_URL = N'tcp://ENTER Container ID:5022',
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
                    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    ),
        N'ENTER Container ID' WITH ( 
		    ENDPOINT_URL = N'tcp://ENTER Container ID:5022', 
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
		    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    );
		
ALTER AVAILABILITY GROUP [ag1] GRANT CREATE ANY DATABASE;

Commands completed successfully.

Total execution time: 00:00:00.145

JOIN SECONDARY node to AG1

In [18]:
ALTER AVAILABILITY GROUP [ag1] JOIN WITH (CLUSTER_TYPE = NONE);
		 
ALTER AVAILABILITY GROUP [ag1] GRANT CREATE ANY DATABASE;


Commands completed successfully.

Total execution time: 00:00:13.796

Add database to AG1

In [3]:
ALTER AVAILABILITY GROUP [ag1] ADD DATABASE [testdb];

Commands completed successfully.

Total execution time: 00:00:00.221

In [4]:
USE testdb;
CREATE TABLE testtable
(id int primary key clustered,
fname varchar(20),
lname varchar(20))

Commands completed successfully.

Total execution time: 00:00:00.088